In [ ]:
# default_exp scraper

# The Guardian Scraper

> Scraping Premier League Previews from the Guardian.

<div style="font-size: 200px">
    
|            Issues                 |          Solutions          |
|------------------------------     |-------------------|
|   4 possible formats for previews(old format, new format,Cup's format and a particular format) |Select the appropriate html tags|
|   Preview titles are not the same ( we can find Squad Sheets or match preview)|Pick only the names of the teams and eliminate the rest|
|   The date of the match is not always available |Pick the preview date|
|   The order of the elements and labels are not the same |Using regex patterns to get information|
|   Missing values for betting odds |We treat the general case separately and we set up specific regex patterns for these particular cases|
|   Odds format is different|We treat the general case separately and we set up specific regex patterns for these particular cases|
|   We can find non-numeric values for Odds like (Evens,evens,Eve)|Replace evens by 1-1|
|   There are some previews that don't have author and text|For previews that have no text, we put 'n/a' (not available)|
|   The existence of previews for the FA CUP,Carabao Cup,Champions league,World Cup|Filter previews by title,link,topic,aside html section and preview text and allow only Premier League previews|
|   We are not sure if the names of the teams are the same as the ones in Opta|Set up a dictionary or check manually to map teams to their IDs|
|When we send many requests, the guardian server blocks your IP address, which is interpreted as a DDOS attack|Do a sleep of a random x seconds between requests or change your IP and work with rotating proxy|
</div >


### Import Libraries and Modules

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
import re
import dateparser
import pandas as pd
import numpy as np
import pymongo
import json
import random 
from mongoengine import * 
from typing import *
from requests_html import HTMLSession
from bs4 import BeautifulSoup
from time import sleep
from datetime import datetime
from os import listdir
from os.path import isfile, join

### Parser Class

##### This class is used to parse pages and has 3 functions:

1- <b> parse_page </b> function:retrieves the html format of a given web page link.

2- <b> store_page_locally </b> function: save a preview page in a specified local folder.

3- <b> get_next_page </b> function: retrieves the link to the next page and determines if it is the last page of previews in order to stop scraping. 


In [ ]:
# export
class Parser:
    """
    A class to represent previews pages parser.

    ...

    Methods
    -------
    parse_page(page_url, session)
        returns the html format of the page.
    store_page_locally(page, page_url)
        save a given page in a local folder.
    get_next_page(page)
        returns the link of the following page and if it's the last page.
    """

    @staticmethod
    def parse_page(page_url: str, session: HTMLSession) -> BeautifulSoup:
        """
            returns the html format of the page.

        Parameters
        ----------
        page_url: str
            the url of the page
        session: requests_html.HTMLSession
            the scraper session

        Returns
        -------
        page: bs4.BeautifulSoup
              the html format of the page

        """
        # Request the url
        request = session.get(page_url)
        # Get the html document of the page
        page = BeautifulSoup(request.text, "html.parser")
        return page

    @staticmethod
    def store_page_locally(page: BeautifulSoup, page_url: str) -> None:
        """
            Save a given page in a local folder.

        Parameters
        ----------
        page: bs4.BeautifulSoup
            the html format of the page
        page_url: str
            the url of the page

        Returns
        -------
        None

        """
        # Get the preview url
        # Delete the "https://www." part
        # Replace "/" by "_"

        page_url = page_url.replace("https://www.", "").replace("/", "_")
        directory_path = ".//previews//"
        file_path = directory_path + page_url + ".html"
        # Create a file and save the html content of the page
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(str(page))
        # Close the file
        file.close()

    @staticmethod
    def get_next_page(page: BeautifulSoup) -> Tuple[str, bool]:
        """
            returns the link of the following page and if it's the last page.

        Parameters
        ----------
        page : bs4.BeautifulSoup
            the html format of the page

        Returns
        -------
        url: str
          the url of the next page
        last_page: bool
          True if it's the last page, False otherwise.

        """
        # If we are at the last page , last_page = True else last_page = False
        last_page = False
        # Pick up the pagination HTML part
        pagination_section = page.find("div", {"class": "pagination__list"})
        # If we don't find the "next" button (it's the last page)
        # We are in the last page
        if not page.find("a", {"rel": "next"}):
            # We pick up the number of the page and we return the link
            html_location = dict({"aria-label": "Current page"})
            page_number = page.find("span", html_location).text
            url = (
                "https://www.theguardian.com/football/series/match-previews?page="
                + page_number
            )
            last_page = True
            return url, last_page
        # If it's not the last page, we pick up the link of the following page
        else:
            url = page.find("a", {"rel": "next"})["href"]
            return url, last_page

### PageExtractor Class

##### This class has five functions for extracting data from a given football preview:

1- <b> get_values_matching_regex </b> returns values that match a regex expression.

&emsp;Because the "Guardian" website has two possible formats, we defined two possible classifiers for the p tags <br>&emsp;containing the information to be extracted.<br>
&emsp;We go through each p section, and if we find the result, we return it; otherwise, a None is returned.<br>
&emsp;The result is a list of tuples, with each tuple representing a value that matches the regex pattern.<br> &emsp;Unsatisfied patterns for regexes that include <b>OR</b> conditions will be empty tuples. That's why you need to get rid of it.

2- <b> extract_teams_names </b> returns the names of the two teams in a football preview.

&emsp;The preview includes team names at the title level.
 <br>&emsp;example:
          &emsp;&emsp;{{Squad Sheets: Team A v Team B}} 
         or &emsp;&emsp;{{Team A v Team B: match preview}} 
         or &emsp;&emsp;{{Team A v Team B: Squad Sheets}}
<br>&emsp;As a result, our strategy is to delete the text preceding or following the names and recover each name <br>&emsp;individually.
<br>&emsp;If we were successful in obtaining the names, they will be returned in a Python dictionary; <br>&emsp;otherwise, the values will be 'n/a'(Not available).

3- <b> extract_text_authors </b>returns the text and author of a football preview.

&emsp;It's difficult to determine the position of the text, but it's almost certainly the block with the most <br>&emsp;characters.
<br>&emsp;To proceed, we store each paragraph and its size in a Python dictionary, and then we take the <br>&emsp;block with the largest size.
<br>&emsp;To be sure, we double-check by only accepting texts with a size greater than 160 because there <br>&emsp;are football previews with no text or author.
<br>&emsp;Furthermore, the author information is always under the text section, more specifically in a <br>&emsp;strong tag, so if the text does not exist, the author is missing as well.
If we were successful in <br>&emsp;obtaining the text and the author, they will be returned in a Python dictionary. Otherwise, the <br>&emsp;values will be 'n/a'(Not available).

4- <b> extract_preview_date </b> returns the date of publication of a football preview.

&emsp;We have distinguished two dates for the date of publication: the first is the date of publication, <br>&emsp;and the second is the date of the most recent modification.
In this sense, we go through the <br>&emsp;section where the two dates are located and take only the first and use 'dateparser' to convert <br>&emsp;the string into a date in "yyyy-mm-dd" format.
If we were successful in obtaining the date, it will <br>&emsp; be returned. Otherwise, the value will be 'n/a'(Not available).

5- <b> extract_match_infos </b> returns a football match information (venue, referee, odds).

&emsp;Here, we'll call the first function <b>get_values_matching_regex</b> , which will allow us to retrieve this<br>&emsp;information by specifying a regex expression for each.<br>&emsp;If this data is not available, the value will be 'n/a'. 

In [ ]:
# export
class PageExtractor:
    """
    A class to represent an information extractor from a football preview.

    ...

    Methods
    -------
    get_values_matching_regex(page, regex)
        return all matched patterns from a preview page.
    extract_teams_names(title)
        returns team names from the preview title.
    extract_text_authors(page)
        returns the text and author of the preview.
    extract_preview_date(page)
        returns the publication date of the preview.
    extract_match_infos(page, venue_regex, referee_regex, odds_regex)
        returns a football match information (venue,referee,odds).
    """

    @staticmethod
    def get_values_matching_regex(
        page: BeautifulSoup, regex: str
    ) -> Union[List[str], None]:
        """
        returns all matched patterns from a preview page.

        Parameters
        ----------
        page: bs4.BeautifulSoup
            the html format of the page
        regex: str
            the regex expression

        Returns
        -------
        result: list of str
          matched values of the regex expression, None otherwise

        """
        # All Information are located in the "p tag" of html
        # We pick up all the p tags
        # some previews in 2009 have a different html tags and classes
        all_p_tags_new_formats = page.find_all("p", {"class": "dcr-bixwrd"})
        all_p_tags_old_format = page.select("div > p")
        # if exist
        if all_p_tags_new_formats:
            paragraphs = all_p_tags_new_formats
        else:
            paragraphs = all_p_tags_old_format

        for paragraph in paragraphs:
            # We pick up the string values located in the paragraph
            # For "odds" information, "Evens" or "Evs" are replaced by 1-1
            pattern_odds = re.compile("Evens|Evs", re.IGNORECASE)
            section = pattern_odds.sub("1-1", paragraph.text)
            # To extract our information regex pattern
            # To ignore case sensitivity we use re.I
            pattern_returned_values = re.compile(regex, re.IGNORECASE)
            # If a regex match is found, we return the list of values.
            # otherwise, an empty array is returned.
            if pattern_returned_values.findall(section):
                matching_result = pattern_returned_values.findall(section)
                # remove empty tuples from the list
                # example of a matching_result value
                # [('12-5', '11-10', '23-10', '', '')]
                result = [element for element in matching_result[0] if element]
                return result
        return None

    @staticmethod
    def extract_teams_names(title: str) -> Dict[str, object]:
        """
        returns team names from the preview title.

        Parameters
        ----------
        title: str
            the title of the preview

        Returns
        -------
        names: dict of object

        """
        # 3 possible formats for previews title
        # For example:
        # {Squad Sheets: Team A v Team B} or
        # {{Team A v Team B : match preview}} or
        # {{Team A v Team B : Squad sheets}}
        # We remove text before or after team names
        pattern = re.compile(
            "Squad Sheets:|: Squad[\s]sheets|Squad sheets|Squad sheet:|: match preview",
            re.IGNORECASE,
        )
        preview_title = pattern.sub("", title).strip()
        # Names are located in the title of the preview
        # Home team
        try:
            home_team = preview_title.split(" v ")[0]
        except Exception as e:
            home_team = None
        # Away team
        try:
            away_team = preview_title.split(" v ")[1].split("\t")[
                0
            ]  # for some preview we find team A v Team B \t date
        except Exception as e:
            away_team = None
        # we return names
        names = dict({"home": home_team, "away": away_team})
        return names

    @staticmethod
    def extract_text_authors(page: BeautifulSoup) -> Dict[str, str]:
        """
        returns the text and author of the preview.

        Parameters
        ----------
        page: bs4.BeautifulSoup
            the html format of the page

        Returns
        -------
        preview_text_author: dict of str

        """
        # Preview may not have text and author,
        # We initialize author and text to 'n/a' (not available),
        author = None
        text = None
        # all items are stored in a p tag
        # Some previews in 2009 have different html tags and classes
        all_p_tags_new_formats = page.find_all("p", {"class": "dcr-bixwrd"})
        all_p_tags_old_format = page.select("div > p")
        # if exist
        if all_p_tags_new_formats:
            all_p_tags = all_p_tags_new_formats
        else:
            all_p_tags = all_p_tags_old_format

        # it's quite difficult to determine which section is the text
        # the length of the text is usually the longest
        # dictionnary to store each p and its length
        length_texts = {}
        for p in all_p_tags:
            section = p.text
            length_texts[p] = len(section)

        # we pick the section with the largest size
        possible_text_section = max(length_texts, key=length_texts.get)
        # We double-check and only select texts with a size greater than 160
        if len(possible_text_section.text) > 160:
            text_section = possible_text_section
            text = text_section.text
            # the author name is located inside the text section
            # it is located in the strong tag
            possible_author_section = text_section.find("strong")
            # for some previews the author information is not found
            # if it's available we take it , else it will be 'n/a'
            if str(possible_author_section) != "None":
                author = possible_author_section.text

        preview_text_author = dict({"text": text, "author": author})
        return preview_text_author

    @staticmethod
    def extract_preview_date(page: BeautifulSoup) -> Union[datetime, str]:
        """
          returns the publication date of the preview.

        Parameters
        ----------
        page: bs4.BeautifulSoup
            the html format of the page

        Returns
        -------
        preview_date: datetime.date
          if not found 'n/a'

        """
        # there are 2 dates for the preview
        # the first is the date of publication
        # the second is the date of the last modification which is hidden
        # we pick only the first one
        try:
            # Some preview in 2009 have different html tags and classes
            html_new_location = dict({"class": "dcr-km9fgb"})
            html_old_location = dict({"itemprop": "datePublished"})
            dates_section_new_format = page.find("div", html_new_location)
            dates_section_old_format = page.find("time", html_old_location)
            if dates_section_new_format:
                dates_section = dates_section_new_format.strings
            else:
                dates_section = dates_section_old_format.strings

            for date in dates_section:
                preview_date = dateparser.parse(date).date()
                break
        except Exception as e:
            preview_date = None

        return preview_date

    @staticmethod
    def extract_match_infos(
        page: BeautifulSoup, venue_regex: str, referee_regex: str, odds_regex: str
    ) -> Dict[str, str]:
        """
          returns a football match information (venue,referee,odds).

        Parameters
        ----------
        page: bs4.BeautifulSoup
            the html format of the page
        venue_regex: str
            venue regex expression
        referee_regex: str
            referee regex expression
        odds_regex: str
            odds regex expression

        Returns
        -------
        match_infos: dict of str

        """
        # Extract venue, referee and odds values
        try:
            venue = PageExtractor.get_values_matching_regex(page, venue_regex)[
                0
            ].strip()
        except Exception as e:
            venue = None
        try:
            referee = PageExtractor.get_values_matching_regex(page, referee_regex)[
                0
            ].strip()
        except Exception as e:
            referee = None

        odds = PageExtractor.get_values_matching_regex(page, odds_regex)

        match_infos = dict({"venue": venue, "referee": referee, "odds": odds})
        return match_infos


### ScrapingTheGuardian Class

##### This class represents a scraper from the "Guardian" website and has 4 functions:

1- <b> calculate_betting_odds </b> returns decimal odds.

&emsp;In this section, we will calculate the odds derived from the football preview.
<br>&emsp;Considering the following example:
<br>&emsp;&emsp; ["9-20","29-5","6-5"] 
<br>&emsp;&emsp;We calculate each sport's rating separately using the following formula:
<br>&emsp;&emsp;&emsp; home = (9/20) + 1 
<br>&emsp;&emsp;&emsp; away = (29/5) + 1
<br>&emsp;&emsp;&emsp; draw = (6/5) + 1
<br>&emsp;If we were successful in obtaining decimal odds, they will be returned in a Python dictionary.<br>&emsp;Otherwise, the values will be 'n/a'(Not available).


2- <b> extract_preview_items </b>returns the entire contents of a football preview.

&emsp;In this section, we will call the functions defined in the PageExtractor class and return a Python dictionary containing all of this information.
<br>&emsp;But first, we use the <b>calculate_betting_odds</b> function to calculate the sports odds for the home team's victory, the away team's victory, and a draw.

"home team","away team","text","author","venue","referee","odds","odds home team","odds away team","odds draw", "preview date","preview_link" are the returned values.

3- <b> save_previews_locally </b> save all browsed previews in a local folder.

&emsp;For a given page, we retrieve all the previews and go through them one by one, taking the link, title, subject, and aside section.
<br>&emsp;if the words "cup" or "champions league" do not belong in these sections, we save the preview in a local folder
<br>&emsp;Otherwise, we move on to the next preview.
<br>&emsp;The <b>store_page_locally</b> function, will be called here to save the preview page.

4- <b> extract_previews_information </b> returns the information of all local previews.

&emsp;In a local folder, we extract the information for each file (preview in html format) by calling the <b> extract_preview_items</b> function <br>&emsp;and then we save the information for each preview in a list.

In [ ]:
# export
class ScrapingTheGuardian:
    """
    A class to represent a scraper from the "Guardian" website.

    ...

    Attributes
    ----------
    session : requests_html.HTMLSession
        a web session
    VENUE_REGEX : str
        venue regex expression
    REFEREE_REGEX : str
        referee regex expression
    ODDS_REGEX : str
        odds regex expression

    Methods
    -------
    calculate_betting_odds(odds)
        returns decimal odds.
    extract_preview_items(page,title)
        returns all information of a football preview.
    save_previews_locally(self,page)
        save all browsed previews in a local folder.
    extract_previews_information(self,folder_path)
        returns all the information of all local previews.

    """

    # venue, referee, odds pattern regex
    # in some previews, all of the information is on the same line.
    VENUE_REGEX = "Venue(.*)Tickets|Venue(.*),|Venue(.*)"
    REFEREE_REGEX = "Referee(.*)This season's|Referee(.*)Last season's|Referee(.*)Odds|Referee(.*)|Ref(.*)Odds"
    # {Odds H 11-8 A 11-8 D 11-8}
    # {Odds Liverpool 11-8 Aston Villa 11-8 Draw 11-8}
    # missing label {Odds H 11-8 11-8 D 11-8}
    # missing value {Odds H 11-8 A 11-8}
    ODDS_REGEX = "Odds[\s]*.*[\s]+(\d{1,3}-\d{1,3})[\s]*.*[\s]+(\d{1,3}-\d{1,3})[\s]*.*[\s]+(\d{1,3}-\d{1,3})|Odds[\s]*.*[\s]+(\d{1,3}-\d{1,3})[\s]*.*[\s]+(\d{1,3}-\d{1,3})"

    def __init__(self):

        # Initialize session to start scraping
        self.session = HTMLSession()

    @staticmethod
    def calculate_betting_odds(odds: list) -> Dict[str, float]:
        """
          returns decimal odds.

        Parameters
        ----------
        odds: list of str
            odds values

        Returns
        -------
        betting_odds: dict of float

        """
        # Initialize betting odds to n/a (not available)
        # Some previews may not include odds
        odds_home = None
        odds_away = None
        odds_draw = None

        if odds is not None:  # If odds exist
            # example of odds:
            # {H 4-6 A 43-10 D 3-1}
            # {liverpool 4-6 Tottenham 43-10 Draw 3-1}
            # {H 4-6 43-10 D 3-1}
            # {H 4-6 A 43-10}
            # The formula will be (4/6)+1 , (43/10)+1 , (3/1)+1
            # Home team odds
            betting_odds_home = odds[0]
            try:
                odds_home = (
                    int(betting_odds_home.split("-")[0])
                    / int(betting_odds_home.split("-")[1])
                ) + 1
            except ZeroDivisionError:
                pass
            # Away team odds
            betting_odds_away = odds[1]
            try:
                odds_away = (
                    int(betting_odds_away.split("-")[0])
                    / int(betting_odds_away.split("-")[1])
                ) + 1
            except ZeroDivisionError:
                pass
            # if we have the normal format of odds
            # we will have 3 parts(odds_home,odds_away,odds_draw)
            if len(odds) == 3:
                # Draw odds
                betting_odds_draw = odds[2]
                try:
                    odds_draw = (
                        int(betting_odds_draw.split("-")[0])
                        / int(betting_odds_draw.split("-")[1])
                    ) + 1
                except ZeroDivisionError:
                    pass

        betting_odds = dict(
            {"odds_home": odds_home, "odds_away": odds_away, "odds_draw": odds_draw}
        )
        return betting_odds

    @staticmethod
    def extract_preview_items(
        page: BeautifulSoup, title: str, link: str
    ) -> Dict[str, object]:
        """
          returns all information of a football preview

        Parameters
        ----------
        page: bs4.BeautifulSoup
            the html format of the page
        title: str
            the title of the preview
        link: str
            the link of the preview

        Returns
        -------
        preview_items: dict of object

        """
        # meth1: extract team names
        names = PageExtractor.extract_teams_names(title)
        # Home team and  Away Team
        home_team = names["home"]
        away_team = names["away"]
        # meth2: extract match infos (venue,referee,odds)
        match_infos = PageExtractor.extract_match_infos(
            page,
            ScrapingTheGuardian.VENUE_REGEX,
            ScrapingTheGuardian.REFEREE_REGEX,
            ScrapingTheGuardian.ODDS_REGEX,
        )
        venue = match_infos["venue"]
        referee = match_infos["referee"]
        odds = match_infos["odds"]
        # meth3: extract text and author of the preview
        text_author = PageExtractor.extract_text_authors(page)
        text = text_author["text"]
        author = text_author["author"]
        # meth4: extract preview date
        preview_date = PageExtractor.extract_preview_date(page)
        # meth5: calculate betting odds
        betting_odds = ScrapingTheGuardian.calculate_betting_odds(odds)
        # Home team betting odds
        odds_home_team = betting_odds["odds_home"]
        # Away team betting odds
        odds_away_team = betting_odds["odds_away"]
        # Draw betting odds
        odds_draw = betting_odds["odds_draw"]
        # Return preview items
        preview_items = dict(
            {
                "home_team": home_team,
                "away_team": away_team,
                "text": text,
                "author": author,
                "venue": venue,
                "referee": referee,
                "odds": odds,
                "odds_home_team": odds_home_team,
                "odds_away_team": odds_away_team,
                "odds_draw": odds_draw,
                "preview_date": preview_date,
                "preview_link": link,
            }
        )
        return preview_items

    def save_previews_locally(self, page: BeautifulSoup) -> None:
        """
          save all browsed previews in local

        Parameters
        ----------
        page: bs4.BeautifulSoup
            the html format of the page

        Returns
        -------
        None

        """
        # We pick all of the match previews on the webpage.
        previews = page.findAll("div", {"class": "fc-item__content"})
        # for each preview we extract its information
        for preview in previews:
            preview_items = {}
            # Pick up the preview link
            preview_link = preview.find("a")["href"]
            print(preview_link)
            # Pick up the match preview page
            preview_page = Parser.parse_page(preview_link, self.session)
            # We need only Premier League Previews
            # To filter previews we need to Find the title of the preview
            # Champions league and Cups are not allowed
            preview_title = preview_page.find("h1").text
            # Check if "cup" or "Champions league" exists in:
            # title, link, preview topic section,preview aside section
            # we pick preview topic
            try:
                preview_topic = preview_page.find("div", {"class": "dcr-lwa3gj"}).text
            except Exception as e:
                # some previews in 2009 have different html tags
                preview_topic = preview_page.find("div", {"class": "submeta"}).text
            # we pick preview_aside
            try:
                preview_aside = preview_page.find(
                    "aside", {"data-gu-name": "title"}
                ).text
            except Exception as e:
                # some previews in 2009 have different html tags
                preview_aside = preview_page.find(
                    "div", {"class": "content__labels"}
                ).text
            # if the preview is not a cup or not for Champions league:
            # we proceed the extraction

            not_premier_league_found = False
            eliminated_matches = ["Champions League", "champions-league", "cup"]
            for word in eliminated_matches:
                # test if the word in the preview title
                if re.search(word, preview_title, re.IGNORECASE):
                    not_premier_league_found = True
                    break
                # test if the word in the preview link
                if re.search(word, preview_link, re.IGNORECASE):
                    not_premier_league_found = True
                    break
                # test if the word in the preview topic
                if re.search(word, preview_topic, re.IGNORECASE):
                    not_premier_league_found = True
                    break
                # test if the word in the preview aside
                if re.search(word, preview_aside, re.IGNORECASE):
                    not_premier_league_found = True
                    break
            # some previews include the type of competition in the text
            # we find FA Cup – Kick-off
            # so we want to eliminate these previews
            cup_in_text = PageExtractor.get_values_matching_regex(
                preview_page, "FA Cup – Kick-off"
            )

            if not not_premier_league_found and not cup_in_text:
                # save preview in a local folder
                Parser.store_page_locally(preview_page, preview_link)

    def extract_previews_information(self, folder_path: str) -> List[Dict[str, object]]:
        """
          returns all the information of all previews saved in a local folder

        Parameters
        ----------
        folder_path: str
            the local folder where previews are saved

        Returns
        -------
        List of(dict of object)

        """
        # get all previews files
        files = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]
        # a list to store previews information
        all_previews_information = []
        # We will proceed with the information extraction for each file (preview).
        for f in files:
            # get the preview path
            preview_path = join(folder_path, f)
            # open the file
            preview = open(preview_path)
            # get the preview link from the preview name
            # The preview's name is its link without the "https://www." and the extension html
            preview_link = (
                preview_path.replace(folder_path, "https://www.")
                .replace("_", "/")
                .replace(".html", "")
            )
            # get the html format of the preview using beautifulSoup
            preview_page = BeautifulSoup(preview, "html.parser")
            preview_title = preview_page.find("h1").text
            # get all information
            preview_infos = ScrapingTheGuardian.extract_preview_items(
                preview_page, preview_title, preview_link
            )
            # store information in the "all_previews_information" list
            all_previews_information.append(preview_infos)
            # just for testing
            print(preview_infos)
            print("-----------------------------------------------")

        return all_previews_information


### PreviewsMapping Class

##### This class represents a mapper from the "Opta" MongoDb database and has 3 functions:

1- <b> get_team_id </b> returns the "opta" ID of a given team.

&emsp;The use of a predefined dictionary containing the teams and their various names facilitates us in matching the team names extracted from the guardian and <br>&emsp;their IDs in the opta database. The use of a dictionary was required because the team names in the previews differ and sometimes use abbreviations or<br>&emsp;nicknames.

2- <b> get_game_id </b>returns the id and the date of a given game.

&emsp;After completing the scraping task and extracting the information, the previews must be matched with their ids in the opta.fixture database.
We will query this <br>&emsp;database by specifying the home team, the away team, the closest gamedate to the preview publication date, and the Premier League competitionId, which  <br>&emsp;is equal to 8 in the database.

3- <b> save_mapped_data </b> save all the mapped previews in a MongoDb collection.

&emsp;For each preview in the data extracted from the Guardian, we will look for the id of the home team and the away team and match it with the <b>opta.fixture</b><br>&emsp;database to get the gameID and gameDate and finally we save it in a MongoDb collection.
<br>&emsp;To complete our mission, we will call the last two functions <b> get_team_id </b> and <b>get_game_id</b>.

In [ ]:
# export
class PreviewsMapping:
    """
    A class to represent a data mapper from a mongo database.

    ...

    Methods
    -------
    get_team_id(team_name, df_teams)
        returns the "opta" ID of a given team.
    get_game_id_date(home_team_id, away_team_id, preview_date)
        returns the id and the date of a given game.
    get_mapped_data(data,df_teams)
        save all the mapped previews in a MongoDb collection.
    """

    @staticmethod
    def get_team_id(team_name: str, df_teams: pd.DataFrame) -> int:
        """
          returns the "opta" ID of a given team.

        Parameters
        ----------
        team_name: str
            the name of a given team
        df_teams: pd.DataFrame
            a dataframe that contains teams and their different names

        Returns
        -------
        int

        """
        # The name of a given team
        # Filter the dictionary
        # If the given team name exists in the dataframe
        # We return its optaID
        # Else we return -1
        team_name = team_name.strip().lower()
        df_filtred = df_teams[
            (df_teams["name"].str.lower() == team_name)
            | (df_teams["shortClubName"].str.lower() == team_name)
            | (df_teams["optaName"].str.lower() == team_name)
            | (df_teams["whoScoredName"].str.lower() == team_name)
            | (df_teams["sofifaName"].str.lower() == team_name)
            | (df_teams["statsName"].str.lower() == team_name)
            | (df_teams["inStatName"].str.lower() == team_name)
            | (df_teams["transfermarktName"].str.lower() == team_name)
            | (df_teams["fotmobName"].str.lower() == team_name)
            | (df_teams["oddsportalName"].str.lower() == team_name)
            | (df_teams["fminsideName"].str.lower() == team_name)
            | (df_teams["nickName1"].str.lower() == team_name)
            | (df_teams["nickName2"].str.lower() == team_name)
            | (df_teams["nickName3"].str.lower() == team_name)
        ]

        if len(df_filtred) > 0:
            return df_filtred["optaId"][df_filtred.index[0]]

        return -1

    @staticmethod
    def get_game_id_date(
        home_team_id: str, away_team_id: str, preview_date: datetime
    ) -> Dict[str, object]:
        """
          returns the id and the date of a given game.

        Parameters
        ----------
        home_team_id: str
            the opta id of a home team.
        away_team_id: str
            the opta id of an away team.

        Returns
        -------
        dict of object

        """
        # Initialize a MongoDb instance with mongoengine
        mongoengine_client = MongoClient.connect("0")
        # Do a MongoDb query
        # Filter data by gameDate,competitionId,
        # homeTeamId, awayTeamId
        # Query
        game_filter = {
            "gameDate": {"$gt": preview_date},
            "competitionId": 8,
            "homeTeamId": int(home_team_id),
            "awayTeamId": int(away_team_id),
        }
        # Get only gameId and gameDate fields
        projection = {"gameId": 1, "gameDate": 1, "_id": 0}
        # Get data
        result = mongoengine_client["opta"]["Fixture"].find_one(
            filter=game_filter, projection=projection
        )
        game_id = None
        game_date = None
        # If there is a match
        # We pick the game ID and date
        if result:
            game_id = result["gameId"]
            game_date = result["gameDate"]

        return dict({"gameId": game_id, "gameDate": game_date})

    @staticmethod
    def save_mapped_data(data: pd.DataFrame, df_teams: pd.DataFrame) -> None:
        """
          returns all the mapped information.

        Parameters
        ----------
        data: pd.DataFrame
            all previews information extracted from the Guardian.
        df_teams: pd.DataFrame
            a dataframe that contains teams and their different names.

        Returns
        -------
        None

        """
        # Copy previews data
        X = data.copy()
        X = X.replace({np.nan: None})
        # For each preview
        # We search home team and away team opta ID's
        # We pick the game ID and date from the opta.Fixture MongoDb collection
        for index, row in X.iterrows():
            # pick the home team name from the preview
            home_team = row["home_team"]
            # pick the away team name from the preview
            away_team = row["away_team"]
            # get their opta ID's
            home_team_id = PreviewsMapping.get_team_id(home_team, df_teams)
            away_team_id = PreviewsMapping.get_team_id(away_team, df_teams)
            # pick the preview date
            preview_date = dateparser.parse(row["preview_date"])
            # get the id and the date of the game
            game = PreviewsMapping.get_game_id_date(
                home_team_id, away_team_id, preview_date
            )
            # connect to our mongoDb cluster
            mongoengine_client = MongoClient.connect("1")
            # preview class
            preview = Previews(
                game_id=game["gameId"],
                home_team=row["home_team"],
                away_team=row["away_team"],
                text=row["text"],
                author=row["author"],
                venue=row["venue"],
                referee=row["referee"],
                odds=row["odds"],
                odds_home_team=row["odds_home_team"],
                odds_away_team=row["odds_away_team"],
                odds_draw=row["odds_draw"],
                game_date=game["gameDate"],
                preview_date=row["preview_date"],
                preview_link=row["preview_link"],
            )
            # Validate and save input raw data
            try:
                preview.validate()
                preview.save()
            except Exception as e:
                print("warning:", e)
                continue


### Previews Class

##### This class represents the extracted previews from the guardian:

We created a class that contains the various attributes to store in order to save every preview information extracted from the Guardian website.
<br>This class ensures that data is stored in a convenient format and type.

In [ ]:
# export
class Previews(Document):
    """
        A class to represent the extracted previews from the guardian.

    ...

    Attributes
    ----------
    game_id : int
        the opta game id
    home_team : str
        home team name
    away_team : str
        away_team name
    text : str
        preview text
    author : str
        preview author
    venue : str
        match venue
    referee : str
        match referee
    odds : str
        betting odds
    odds_home_team : float
        decimal betting odds for home team
    odds_away_team : float
        decimal betting odds for away team
    odds_draw : float
        decimal betting odds for draw
    game_date : datetime
        the date of the match
    preview_date : datetime
        the date of the preview
    preview_link : str
        the Guardian preview link

    """

    game_id = IntField()
    home_team = StringField()
    away_team = StringField()
    text = StringField()
    author = StringField()
    venue = StringField()
    referee = StringField()
    odds = StringField()
    odds_home_team = FloatField()
    odds_away_team = FloatField()
    odds_draw = FloatField()
    game_date = DateTimeField()
    preview_date = DateTimeField()
    preview_link = StringField()


### MongoCLient Class

##### This class represents a MongoDb Client and has 2 functions:

1- <b> find_credentials </b> returns the MongoDb credentials stored in a local file.


2- <b> connect </b>returns the MongoDb instance to connect to a given cluster.

There are two database URIs in the credentials file, the first of which is for the "OPTA" Database.
<br>The second is for another database that was created to store extracted previews.


In [ ]:
# export
class MongoClient:
    """
    A class to represent a MongoDb client.

    ...

    Attributes
    ----------
    CREDENTIALS_PATH : str
        the file path of the MongoDb credentials

    Methods
    -------
    find_credentials()
        returns MongoDb credentials stored in a local file.
    connect(index)
        returns the mongoDb instance.
    """

    # the file path of the MongoDb credentials
    CREDENTIALS_PATH = "//home//meherkh//secrets//credentials.json"

    @staticmethod
    def find_credentials() -> dict:
        """
          returns MongoDb credentials stored in a local file.

        Returns
        -------
        dict

        """
        # open file and extract the json fields
        with open(MongoClient.CREDENTIALS_PATH) as credentials:
            mongo_credentials = json.load(credentials)
            return mongo_credentials

    @staticmethod
    def connect(index: str) -> pymongo.mongo_client.MongoClient:
        """
          returns the mongoDb instance.
          
        Parameters
        ----------
        index: str
            the index of the cluster

        Returns
        -------
        pymongo.mongo_client.MongoClient

        """
        # Initialize a MongoDb instance with mongoengine
        disconnect()
        DB_URI = MongoClient.find_credentials()["DB_URI"][index]
        mongoengine_client = connect(host=DB_URI)
        return mongoengine_client


## USE CASE


### Scraping one page and save its previews in local

In [ ]:
url = "https://www.theguardian.com/football/series/match-previews"
scraper = ScrapingTheGuardian()
page = Parser.parse_page(url,scraper.session)
previews_infos = scraper.save_previews_locally(page)

https://www.theguardian.com/football/2022/jan/22/crystal-palace-liverpool-match-preview-premier-league
https://www.theguardian.com/football/2022/jan/22/leicester-brighton-match-preview-premier-league
https://www.theguardian.com/football/2022/jan/22/arsenal-burnley-match-preview-premier-league
https://www.theguardian.com/football/2022/jan/22/chelsea-tottenham-match-preview-premier-league
https://www.theguardian.com/football/2022/jan/21/manchester-united-west-ham-match-preview-premier-league
https://www.theguardian.com/football/2022/jan/21/southampton-manchester-city-match-preview-premier-league
https://www.theguardian.com/football/2022/jan/21/brentford-wolves-match-preview-premier-league
https://www.theguardian.com/football/2022/jan/21/everton-aston-villa-match-preview-premier-league
https://www.theguardian.com/football/2022/jan/21/leeds-newcastle-match-preview-premier-league
https://www.theguardian.com/football/2022/jan/15/west-ham-leeds-match-preview-premier-league
https://www.theguar

### Scraping all pages and save in local

url = "https://www.theguardian.com/football/series/match-previews"
scraper = ScrapingTheGuardian()
last_page = False
while not last_page : 
    print(url)
    page = Parser.parse_page(url,scraper.session)
    previews_infos = scraper.save_previews_locally(page)
    url, last_page = Parser.get_next_page(page)
    time = random.randint(2, 60)
    print("timer")
    print(time) 
    # wait time seconds
    sleep(time)

### Extract information from local previews

In [ ]:
local_folder = ".//previews//"
scraper = ScrapingTheGuardian()
all_previews_information = scraper.extract_previews_information(local_folder)

{'home_team': 'Crystal Palace', 'away_team': 'Liverpool', 'text': 'Liverpool have already dispelled most of the doubts about how they would cope while Mohamed Salah and Sadio Mané are at the Africa Cup of Nations. They are yet to lose without their star forwards but Jürgen Klopp knows there is no room for error if they are to maintain any hope of reeling in Manchester City. Crystal Palace have lost their past nine league meetings with Liverpool by an aggregate score of 27-5 but will be mightily relieved not to be facing Salah and Mané, as they have contributed 17 of those goals between them. Palace are boosted by Jordan Ayew’s early return following Ghana’s Afcon exit. Stephen Hollis', 'author': 'Stephen Hollis', 'venue': 'Selhurst Park', 'referee': 'Kevin Friend', 'odds': ['5-1', '8-13', '10-3'], 'odds_home_team': 6.0, 'odds_away_team': 1.6153846153846154, 'odds_draw': 4.333333333333334, 'preview_date': datetime.date(2022, 1, 22), 'preview_link': 'https://www.theguardian.com/football/

{'home_team': 'Liverpool', 'away_team': 'Brentford', 'text': 'Albert Stubbins, the only footballer to feature on the cover of the Beatles’ Sgt. Pepper’s album, scored the winner when Liverpool last hosted Brentford at Anfield in the league. That was in October 1946. Seventy-six years on, Jürgen Klopp takes charge of Liverpool for the 350th time boasting the highest win percentage (60.74%) of any manager in the club’s league history. With Manchester City or Chelsea, or both, guaranteed to drop points when they meet on Saturday, Klopp will be desperately seeking an improvement on Thursday’s display against Arsenal to take advantage. Andy Hunter', 'author': 'Andy Hunter', 'venue': 'Anfield', 'referee': 'Jon Moss', 'odds': ['1-3', '21-2', '5-1'], 'odds_home_team': 1.3333333333333333, 'odds_away_team': 11.5, 'odds_draw': 6.0, 'preview_date': datetime.date(2022, 1, 15), 'preview_link': 'https://www.theguardian.com/football/2022/jan/15/liverpool-brentford-match-preview-premier-league'}
------

{'home_team': 'Chelsea', 'away_team': 'Tottenham', 'text': 'The dynamic has changed before Chelsea and Tottenham meet for the third time this month. Chelsea dominated their rivals in the Carabao Cup semi-finals but Thomas Tuchel’s side have stuttered in the league and Spurs are on a high after their dramatic win at Leicester. Harry Kane is scoring again and Antonio Conte, who remains unbeaten in the league since his arrival in north London, will sense a chance to drag his old team into a battle for a place in the top four. Jacob Steinberg', 'author': 'Jacob Steinberg', 'venue': 'Stamford Bridge', 'referee': 'Paul Tierney', 'odds': ['13-18', '9-2', '13-4'], 'odds_home_team': 1.7222222222222223, 'odds_away_team': 5.5, 'odds_draw': 4.25, 'preview_date': datetime.date(2022, 1, 22), 'preview_link': 'https://www.theguardian.com/football/2022/jan/22/chelsea-tottenham-match-preview-premier-league'}
-----------------------------------------------
{'home_team': 'Chelsea', 'away_team': 'Liverpool

In [ ]:
data = pd.DataFrame(all_previews_information)

In [ ]:
data

,home_team,away_team,text,author,venue,referee,odds,odds_home_team,odds_away_team,odds_draw,preview_date,preview_link
0,Crystal Palace,Liverpool,Liverpool have already dispelled most of the d...,Stephen Hollis,Selhurst Park,Kevin Friend,"[5-1, 8-13, 10-3]",6.000000,1.615385,4.333333,2022-01-22,https://www.theguardian.com/football/2022/jan/...
1,Newcastle,Watford,Moussa Sissoko returns to Newcastle in Watford...,Louise Taylor,St James’ Park,Paul Tierney,"[19-17, 27-10, 27-10]",2.117647,3.700000,3.700000,2022-01-14,https://www.theguardian.com/football/2022/jan/...
2,Manchester United,West Ham,This is a genuine six-pointer in which Manches...,Jamie Jackson,Old Trafford,Jon Moss,"[10-11, 3-1, 11-4]",1.909091,4.000000,3.750000,2022-01-21,https://www.theguardian.com/football/2022/jan/...
3,Manchester City,Chelsea,Manchester City would surely finish off Chelse...,Jamie Jackson,Etihad Stadium,Craig Pawson,"[8-11, 22-5, 3-1]",1.727273,5.400000,4.000000,2022-01-14,https://www.theguardian.com/football/2022/jan/...
4,Arsenal,Burnley,Arsenal have no excuses if their dismal form i...,Graham Searles,Emirates Stadium,David Coote,"[2-5, 9-1, 4-1]",1.400000,10.000000,5.000000,2022-01-22,https://www.theguardian.com/football/2022/jan/...
5,Brentford,Wolves,Wolves have arguably been the surprise package...,Stephen Hollis,Brentford Community Stadium,Peter Bankes,"[9-4, 13-8, 2-1]",3.250000,2.625000,3.000000,2022-01-21,https://www.theguardian.com/football/2022/jan/...
6,Everton,Aston Villa,The second caretaker coming of Duncan Ferguson...,Andy Hunter,Goodison Park,Craig Pawson,"[7-4, 17-10, 40-17]",2.750000,2.700000,3.352941,2022-01-21,https://www.theguardian.com/football/2022/jan/...
7,Wolves,Southampton,Southampton were excellent in their 4-1 stroll...,Graham Searles,Molineux,Michael Salisbury,"[11-8, 5-2, 9-4]",2.375000,3.500000,3.250000,2022-01-14,https://www.theguardian.com/football/2022/jan/...
8,Liverpool,Brentford,"Albert Stubbins, the only footballer to featur...",Andy Hunter,Anfield,Jon Moss,"[1-3, 21-2, 5-1]",1.333333,11.500000,6.000000,2022-01-15,https://www.theguardian.com/football/2022/jan/...
9,West Ham,Leeds,West Ham regained fourth place by beating Norw...,Jacob Steinberg,London Stadium,Mike Dean,"[4-6, 43-10, 3-1]",1.666667,5.300000,4.000000,2022-01-15,https://www.theguardian.com/football/2022/jan/...


In [ ]:
data.home_team.unique()

array(['Crystal Palace', 'Newcastle', 'Manchester United',
       'Manchester City', 'Arsenal', 'Brentford', 'Everton', 'Wolves',
       'Liverpool', 'West Ham', 'Aston Villa', 'Leicester', 'Leeds',
       'Norwich', 'Southampton', 'Tottenham', 'Chelsea'], dtype=object)

In [ ]:
data.away_team.unique()

array(['Liverpool', 'Watford', 'West Ham', 'Chelsea', 'Burnley', 'Wolves',
       'Aston Villa', 'Southampton', 'Brentford', 'Leeds',
       'Manchester United', 'Brighton', 'Newcastle', 'Everton',
       'Manchester City', 'Arsenal', 'Tottenham'], dtype=object)

In [ ]:
data.to_csv(".//datasets//previews.csv",index=False)

## Part 2 : Mapping

In [ ]:
# a dictionary that contains the teams and their different names.
df_dict = pd.read_csv(".//datasets//final_data.csv")
# extracted previews
df_guardian = pd.read_csv(".//datasets//previews.csv")

In [ ]:
df_dict.head()

,ID,optaId,name,symid,shortClubName,optaName,whoScoredName,sofifaName,statsName,inStatName,transfermarktName,fotmobName,oddsportalName,fminsideName,nickName1,nickName2,nickName3
0,9e78bbc137fd00c66162080bc9e987e67297643dc50616...,3,Arsenal,ARS,Arsenal,Arsenal,Arsenal,Arsenal,Arsenal,Arsenal FC,Arsenal FC,Arsenal,Arsenal,Arsenal,Arsenal FC,Arsenal Football Club,The Gunners
1,0ef9883721814dd09038659130c61c76f18976cb7b8e86...,47,Portsmouth,POR,Portsmouth,Portsmouth,Portsmouth,Portsmouth,Portsmouth,Portsmouth,Portsmouth FC,NaN,NaN,NaN,Portsmouth FC,Portsmouth Football Club,Pompey
2,eb89c068ca204a72408360450847a990c97c5b5ff0ec9f...,110,Stoke City,STK,Stoke,Stoke City,Stoke,Stoke City,Stoke City,Stoke,Stoke City,Stoke,Stoke,Stoke,Stoke City FC,Stoke City Football Club,The Potters
3,c1a486f8ca465e58b6301f038e754058986187d454110c...,56,Sunderland,SUN,Sunderland,Sunderland,Sunderland,Sunderland,Sunderland,Sunderland AFC,Sunderland AFC,Sunderland,NaN,NaN,Sunderland FC,Sunderland Association Football Club,Sunderland A.F.C.
4,0db353094ccf93e0005cf378ea862b56e77cacc57b7c5e...,111,Wigan Athletic,WIG,Wigan,Wigan Athletic,Wigan,Wigan Athletic,Wigan Athletic,Wigan Athletic,Wigan Athletic,Wigan,Wigan,NaN,Wigan Athletic FC,Wigan Athletic Football Club,The Latics The Tics


In [ ]:
df_guardian.head()

,home_team,away_team,text,author,venue,referee,odds,odds_home_team,odds_away_team,odds_draw,preview_date,preview_link
0,Crystal Palace,Liverpool,Liverpool have already dispelled most of the d...,Stephen Hollis,Selhurst Park,Kevin Friend,"['5-1', '8-13', '10-3']",6.000000,1.615385,4.333333,2022-01-22,https://www.theguardian.com/football/2022/jan/...
1,Newcastle,Watford,Moussa Sissoko returns to Newcastle in Watford...,Louise Taylor,St James’ Park,Paul Tierney,"['19-17', '27-10', '27-10']",2.117647,3.700000,3.700000,2022-01-14,https://www.theguardian.com/football/2022/jan/...
2,Manchester United,West Ham,This is a genuine six-pointer in which Manches...,Jamie Jackson,Old Trafford,Jon Moss,"['10-11', '3-1', '11-4']",1.909091,4.000000,3.750000,2022-01-21,https://www.theguardian.com/football/2022/jan/...
3,Manchester City,Chelsea,Manchester City would surely finish off Chelse...,Jamie Jackson,Etihad Stadium,Craig Pawson,"['8-11', '22-5', '3-1']",1.727273,5.400000,4.000000,2022-01-14,https://www.theguardian.com/football/2022/jan/...
4,Arsenal,Burnley,Arsenal have no excuses if their dismal form i...,Graham Searles,Emirates Stadium,David Coote,"['2-5', '9-1', '4-1']",1.400000,10.000000,5.000000,2022-01-22,https://www.theguardian.com/football/2022/jan/...


In [ ]:
X = df_guardian.copy()
PreviewsMapping.save_mapped_data(X,df_dict)


/home/meherkh/anaconda3/envs/Real_Analytics/lib/python3.7/site-packages/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)
